In [214]:
using Plots, Distributions, ColorSchemes, Statistics, Clustering, LinearAlgebra,  GeometryBasics, VoronoiCells
default(size=(800,800))

## Demandas de combustible

In [189]:
nloc=100
loc = rand(Uniform(-1,1), nloc, 2)
demandas = rand(Exponential(1),nloc);

In [190]:
scatter(loc[:,1],loc[:,2], 
    markersize=30*sqrt.(demandas), 
    marker=:circle, 
    colorbar=:none,
    color=:teal,
    legend=:none, 
    markeralpha=0.4, 
    aspectratio=:equal, 
    ticks=false, 
    axis=false,
    title="Demandas de kilómetros en sitios de medida (estaciones)",
    background=RGB(0.9,0.9,0.95)
)
savefig("demandas.png")

In [198]:
m=5
km = kmeans(loc', m, weights=demandas)
centers = km.centers'
sigma = [sqrt.(tr(cov(loc[km.assignments.==i,:]))) for i=1:m];

In [199]:
function rbf(x;weigths=[1],centers=[0,0],sigma2=[1])
    
    @assert length(w)==length(mu)==length(sigma2)>0 #chequeo consistencia de n>0
    n=length(w)

    @assert length(x)==length(mu[1]) #chequeo que los centros tengan la dim de x.
    d=length(x)

    
    kernels = [exp(-(x-mu[j])'*(x-mu[j])/sigma2[j]) for j=1:n]
    return sum(w.*kernels)
    
end

rbf (generic function with 1 method)

In [200]:
mu = [[centers[i,1],centers[i,2]] for i=1:m]
sigma2 = sigma.^2
w = ones(length(sigma2))

x1=(-1:.01:1)
x2=(-1:.01:1)

x=[[a,b] for b in x2, a in x1] ##el orden de b y a importa no entendí por qué.

z = rbf.(x, weigths=w,centers=mu,sigma2=sigma2)

heatmap(x1,x2,z,
    c=cgrad([:white, :teal]),
    axis=false, 
    ticks=false, 
    lims=(-1,1),
    colorbar=:none, 
    aspectratio=:equal, 
    legend=:none,    
    background=RGB(0.9,0.9,0.95),
    title="Ajuste por núcleos RBF")

scatter!(loc[:,1],loc[:,2], 
    markersize=30*sqrt.(demandas), 
    marker=:circle, 
    color=:gray,
    markeralpha=0.1, 
)
savefig("ajuste.png")

In [201]:
function sample_rbf(N,rbf_func,xlims,ylims,cota=2.0)
    
    v=Array{Float64}(undef,N,2)
    n=0
    
    while(n<N)
        
        u1 = rand(Uniform(xlims[1],xlims[2]))
        u2 = rand(Uniform(ylims[1],ylims[2]))

        u = rand(Uniform())
        
        val  = rbf_func([u1,u2])
        
        if val/cota > u
            #accept
            n=n+1
            v[n,:] = [u1,u2]
        end
    end
    return v
end

sample_rbf (generic function with 2 methods)

In [202]:
points = sample_rbf(10000,x->rbf(x,weigths=w,centers=mu,sigma2=sigma2),(-1,1), (-1,1))
scatter(points[:,1], points[:,2], 
    aspect_ratio=:equal, 
    ticks=false, 
    axis=false, 
    lims=(-1,1), 
    label=:none, 
    ms=1,
    markerstrokewidth=0,
    background=RGB(0.9,0.9,0.95),
    title="Simulación de demanda")
savefig("sim.png")

In [203]:
n=10
res = kmeans(points',n)
cargadores = (res.centers)';

In [204]:
heatmap(x1,x2,z, 
    aspect_ratio=:equal, 
    c=c=cgrad([:white, :teal]),
    axis=false, ticks=false, lims=(-1,1),
    colorbar=:none, 
    alpha=0.5,
    background=RGB(0.9,0.9,0.95)
)

scatter!(cargadores[:,1], cargadores[:,2], legend=:none, color=:teal, title="Ubicación de cargadores")
savefig("cargadores.png")

In [287]:
Na=200
Nc=n
rect = Rectangle(Point2(-1.0, -1.0), Point2(1.0, 1.0))

#autos = sample_rbf(Na,x->rbf(x,weigths=w,centers=mu,sigma2=sigma2),(-1,1), (-1,1))
autos = rand(Uniform(-1,1),Na,2)

d(x,y) = sqrt(sum((x-y).^2))
W = [d(autos[i,:],cargadores[j,:]) for i=1:Na,j=1:Nc]

attachs = zeros(Na)

for i=1:Na
    _,idx = findmin(W[i,:])
    attachs[i] = idx
end
attachs=Int64.(attachs);

X = Point2{Float64}[]
for i=1:n
    aux = Point2(cargadores[i,1],cargadores[i,2])
    push!(X,aux)
end
tess = voronoicells(X, rect);


scatter(autos[:,1],autos[:,2], xlims=(-1,1), ylims=(-1,1), aspectratio=:equal, marker=:plus)
scatter!(cargadores[:,1], cargadores[:,2], 
    legend=:none, 
    colorbar=:none, 
    background=RGB(0.9,0.9,0.95), axis=false,ticks=false,
    color=:teal,
)

for i=1:Na
    plot!([autos[i,1],cargadores[attachs[i],1]], [autos[i,2],cargadores[attachs[i],2]], color=:black, alpha=0.2)
end

plot!(tess, alpha=0.2, color=:teal, linewidth=2, title="Celdas de Voronoi (menor distancia euclidiana)")

savefig("voronoi.png")

In [312]:
using JuMP, Gurobi

model = Model(Gurobi.Optimizer)

@variable(model,pi[1:Nc,1:Na]>=0)

con1 = @constraint(model, sum(pi,dims=2).<= 20*ones(Nc))
con2 = @constraint(model, sum(pi,dims=1).== ones(1,Na))

@objective(model,Min,sum(pi.*W'))

optimize!(model)

sol = value.(pi)

attachs=zeros(Na)
for i=1:Na 
    _,idx = findmax(sol[:,i])
    attachs[i] = idx
end

attachs = Int64.(attachs);

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 210 rows, 2000 columns and 4000 nonzeros
Model fingerprint: 0x6e0be7b0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.00s
Presolved: 210 rows, 2000 columns, 4000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1929497e+01   3.300000e+01   0.000000e+00      0s
      52    5.5951278e+01   0.000000e+00   0.000000e+00      0s

Solved in 52 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.595127787e+01

User-callback calls 90, time in user-callback 0.00 sec


In [314]:
scatter(autos[:,1],autos[:,2], xlims=(-1,1), ylims=(-1,1), aspectratio=:equal, marker=:plus)
scatter!(cargadores[:,1], cargadores[:,2], 
    legend=:none, 
    colorbar=:none, 
    background=RGB(0.9,0.9,0.95), axis=false,ticks=false,
    color=:teal,
)

for i=1:Na
    plot!([autos[i,1],cargadores[attachs[i],1]], [autos[i,2],cargadores[attachs[i],2]], color=:black, alpha=0.2)
end
plot!(tess, alpha=0.2, color=:teal, linewidth=2, title="Celdas de Voronoi (menor distancia euclidiana)")

plot!(title="Asignación con congestión")
savefig("congestion.png")

In [316]:
using JuMP, Gurobi

model = Model(Gurobi.Optimizer)

@variable(model,pi[1:Nc,1:Na]>=0)
@variable(model,s[1:Nc]>=0)


con1 = @constraint(model, sum(pi,dims=2).<= s)
con2 = @constraint(model, sum(pi,dims=1).== ones(1,Na))
con3 = @constraint(model, s.<= 25*ones(Nc,1))

@objective(model,Min,sum(pi.*W') + sum(s))

optimize!(model)

sol = value.(pi)

attachs=zeros(Na)
for i=1:Na 
    _,idx = findmax(sol[:,i])
    attachs[i] = idx
end

attachs = Int64.(attachs);
dual.(con1)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 220 rows, 2010 columns and 4020 nonzeros
Model fingerprint: 0xe7c9d078
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 210 rows, 2010 columns, 4010 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1929497e+01   1.000000e+02   0.000000e+00      0s
      69    2.5313606e+02   0.000000e+00   0.000000e+00      0s

Solved in 69 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.531360619e+02

User-callback calls 124, time in user-callback 0.00 sec


10×1 Matrix{Float64}:
 -1.0
 -1.1669871236011773
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0152307838085077
 -1.1741432338364257
 -1.0

In [322]:
scatter(autos[:,1],autos[:,2], xlims=(-1,1), ylims=(-1,1), aspectratio=:equal, marker=:plus)
scatter!(cargadores[:,1], cargadores[:,2], 
    legend=:none, 
    colorbar=:none, 
    background=RGB(0.9,0.9,0.95), axis=false,ticks=false,
    color=:teal,
)

for i=1:Na
    plot!([autos[i,1],cargadores[attachs[i],1]], [autos[i,2],cargadores[attachs[i],2]], color=:black, alpha=0.2)
end
plot!(tess, alpha=0.2, color=:teal, linewidth=2, title="Celdas de Voronoi (menor distancia euclidiana)")

scatter!(cargadores[[2,8,9],1], cargadores[[2,8,9],2], 
    ms = 10,
    background=RGB(0.9,0.9,0.95), axis=false,ticks=false,
    color=:teal)

plot!(title="Asignación con congestión")
savefig("congestion2.png")